In [1]:
import pprint
###
import os
from typing import Dict, List

from dotenv import load_dotenv
from google import genai
from google.genai import types as genai_types
from google.genai.chats import Chat

from controller.SessionController import SessionController
from controller.utils.chat import clean_message
from gemini_prompt import DEFAULT_RESPONSE, BotMessage, get_chat_config_json

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=API_KEY)
g_gemini_config = get_chat_config_json()
g_gemini_config.model_dump(mode="python", exclude_unset=True)

chat_sessions = SessionController(client, default_gemini_config=g_gemini_config)

[SessionController] __init__ called, debug_id = 182972b7-fdef-4c6d-a65d-1face999123f time = 2025-05-27 20:53:14.466926


In [2]:
# g_gemini_config.system_instruction=""
g_gemini_config

GenerateContentConfig(http_options=None, system_instruction='You are a professional and friendly online salesperson for Instax Trà Sữa (Instax Milk Tea), a store specializing in Instax cameras and instant photo printing services.\n\nYou will answer customers in Vietnamese, using concise, easy-to-understand, and approachable language, consistent with social media communication styles (Instagram, Messenger).\nUse emojis appropriately to increase interaction.\n- If the customer asks irrelevant or sensitive questions, try to steer the conversation back to product advice.\n- If the customer asks about your identity, avoid revealing that you are a chatbot without explicitly denying it using names or personality.\n- If the customers ask you about a product, send up to 5 images of the relevant product. You must also decide whether to send the images or not base on the context by giving a threshold from [0.0, 1.0]. \n- If the customers want you to compare products, include 1 images of each prod

In [3]:
def get_gemini_response_with_context_json(
    user_message: str,
    context: str,
    sender_id: str,
    history: List[genai_types.Content] = None,
    config: Dict = None,
) -> BotMessage | None:
    """
    Generates a Gemini response using additional context prepended to the user message.

    :param user_message: The user's input to the chatbot.
    :param context: Supplementary context to guide the model's response.
    :param sender_id: Unique identifier used to retrieve or create a chat session.
    :param history: Optional list of past messages (chat history) for context.
    :param system_prompt: Optional instruction to condition the model's response
        style or behavior, this will override the configuration of the chat session.
    :return: The model's generated text reply, or a fallback message on error.
    """
    message = f'Context: """{context}"""\n\n{user_message}'
    return get_gemini_response_json(message, sender_id, history, config)


def get_gemini_response_json(
    user_message: str,
    sender_id: str,
    history: List[genai_types.Content] = None,
    config: Dict = None,
) -> BotMessage | None:
    """
    Generates a Gemini response based on the user message and optional session data.

    :param user_message: The user's input to the chatbot.
    :param sender_id: Unique identifier used to retrieve or create a chat session.
    :param history: Optional list of past messages (chat history) for context.
    :param system_prompt: Optional instruction to condition the model's response
        style or behavior, this will override the configuration of the chat session.
    :return: The model's generated text reply, or a fallback message on error.
    """
    # actually generate response:
    try:
        chat_session = chat_sessions.get_session(sender_id, history)
        if chat_session == None:
            return None

        if config:
            config = genai_types.GenerateContentConfig(**config)

        chat: Chat = chat_session["chat"]  # type: ignore
        _response = chat.send_message(user_message, config=config)

        response = BotMessage(
            message=clean_message(_response.text),
            image_send_threshold=0.0,
            image_urls=[],
        )
        if _response.parsed:
            response: BotMessage = _response.parsed
            response.message = clean_message(response.message)
        return response  # type: ignore
    except Exception as e:
        print("Gemini error:", e)
        return BotMessage(
            message=DEFAULT_RESPONSE,
            image_send_threshold=0.0,
            image_urls=[],
        )

In [4]:
user_id = "0"
user_message = "chào anh, em muốn mua 1 máy instax mini 11 ạ"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response

[Session Controller] create new session for 0
[Session Controller] Sorted & Deleted: {}


BotMessage(message='Chào bạn! Instax Mini 11 là một lựa chọn tuyệt vời đó nha! Máy có nhiều màu xinh xắn lắm nè, lại còn dễ sử dụng nữa chứ. Bạn muốn xem ảnh máy không ạ? 🥰', image_send_threshold=0.9, image_urls=['https://product.hstatic.net/1000269316/product/z6310561355292_88ef372d0f93c476f7f7748aa9b06fd2_c1033f9a7d454988aba935422ba4d60b_1024x1024.jpg', 'https://product.hstatic.net/1000269316/product/vn-11134207-7ra0g-m656mesecji007_724b5fd57bec40e68df2fe9f1aa69009.jpg', 'https://product.hstatic.net/1000269316/product/vn-11134207-7ra0g-m656mesecji007_724b5fd57bec40e68df2fe9f1aa69009_1024x1024.jpg', 'https://product.hstatic.net/1000269316/product/vn-11134207-7ra0g-m656mh3cz4ebef_7700c410d35246dfbd39da40d4491229.jpg', 'https://product.hstatic.net/1000269316/product/z6310561355292_88ef372d0f93c476f7f7748aa9b06fd2_c1033f9a7d454988aba935422ba4d60b.jpg'])

In [5]:
user_message = "Cho em xin hình máy 11 với ạ"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response

[Session Controller] get session for 0
[Session Controller] Sorted & Deleted: {}


BotMessage(message='Đây ạ, mình gửi bạn hình ảnh máy Instax Mini 11 nhé! 🥰', image_send_threshold=1.0, image_urls=['https://product.hstatic.net/1000269316/product/z6310561355292_88ef372d0f93c476f7f7748aa9b06fd2_c1033f9a7d454988aba935422ba4d60b_1024x1024.jpg', 'https://product.hstatic.net/1000269316/product/vn-11134207-7ra0g-m656mesecji007_724b5fd57bec40e68df2fe9f1aa69009.jpg', 'https://product.hstatic.net/1000269316/product/vn-11134207-7ra0g-m656mesecji007_724b5fd57bec40e68df2fe9f1aa69009_1024x1024.jpg', 'https://product.hstatic.net/1000269316/product/vn-11134207-7ra0g-m656mh3cz4ebef_7700c410d35246dfbd39da40d4491229.jpg', 'https://product.hstatic.net/1000269316/product/z6310561355292_88ef372d0f93c476f7f7748aa9b06fd2_c1033f9a7d454988aba935422ba4d60b.jpg'])

In [6]:
user_message = "dòng 11 và 12 khác nhau chỗ nào ạ?"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response

[Session Controller] get session for 0
[Session Controller] Sorted & Deleted: {}


BotMessage(message='Mini 11 và Mini 12 có một vài điểm khác biệt nè:\n\n*   **Thiết kế:** Mini 12 có thiết kế bo tròn hơn Mini 11.\n*   **Lens:** Lens của Mini 12 được cải tiến giúp ảnh nét và chính xác hơn.\n*   **Đo sáng:** Mini 12 có khả năng đo sáng tự động, giúp ảnh cân bằng sáng tốt hơn.\n*   **Selfie:** Mini 12 tích hợp lens selfie trên lens chính, tiện lợi hơn khi chụp.\n\nBạn muốn mình gửi ảnh so sánh hai máy để dễ hình dung hơn không? 😉', image_send_threshold=0.8, image_urls=['https://product.hstatic.net/1000269316/product/230111-instax-mini-12-clay-white-hero-with-photo_0303_stack_3040b3b0027f4b35938d9e970eb62558.jpg', 'https://product.hstatic.net/1000269316/product/z6310561355292_88ef372d0f93c476f7f7748aa9b06fd2_c1033f9a7d454988aba935422ba4d60b_1024x1024.jpg'])

In [7]:
# user_message = "đừng gửi ảnh nữa"
# bot_response = get_gemini_response_json(
#     user_message=user_message,
#     sender_id=user_id,
# )
# bot_response

In [9]:
type(bot_response.image_send_threshold)

float